# RPGゲームを作ろう2

- RPGゲームを作ろう1で作成したゲームを改良を加えよう
(このコード自体はすでに完成しているので、一度実行してみましょう)


#### (改良point)
1. キャラのステータスを変更する(初級)
2. スキルを変更する。(中級~上級)
3. バインド状態を作成する(上級 ~ 鬼)
4. キャラタイプ相性を作成(上級 ~ 鬼)
5. その他工夫

---

In [4]:
class Character:
    def __init__(self, name, hp, power, mp):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.power = power
        self.mp = mp
        self.max_mp = mp
    
    #全体攻撃に対応 (targetはリストを想定)
    def attack(self, target):
        results = []
        
        #単体攻撃との互換性の保持
        if not isinstance(target, list):
            target = [target]

        for other in target:
            #ダメージにわずかな変動を加える
            damage = self.power + random.randint(-2, 2) 
            other.hp -= damage
            
            if other.hp <= 0:
                other.hp = 0
                results.append(f"{self.name}が{other.name}を攻撃！{other.name}のHPは{other.hp}になった。\n{other.name}は{self.name}に倒された!\n")
            else:
                results.append(f"{self.name}が{other.name}を攻撃！{other.name}のHPは{other.hp}になった。\n")
        
        return "".join(results)

    
    def special_attack(self, target):
        #必殺技は子クラスで実装
        return f"{self.name}には必殺技が定義されていません。"

    # 生存チェック
    def is_alive(self):
        return self.hp > 0

In [37]:
#勇者パーティーのクラスの定義
import copy
import random
import math

HERO = 'hero'
HEALER = 'healer'
MAGICIAN = 'magician'
GARDIAN = 'gardian'

class Explorer(Character):
    def __init__(self, name, hp, power, mp, role, level=1, leader=False):
        super().__init__(name, hp, power, mp)
        self.role = role
        self.level = level
        self.leader = leader
        self._initial_state = copy.deepcopy(self.__dict__)

    #Characterクラスのspecial_attackをオーバーライド
    def special_attack(self, target):
        results = []
        
        #ターゲットが単体の場合も考慮し、リストに変換
        if not isinstance(target, list):
            target = [target]
        
        if self.mp < 3:
            return f"{self.name}: MPが足りない！ (現在のMP: {self.mp})"
        
        self.mp -= 3 
        
        #-----------------------------
        #勇者の必殺技: 閃空斬り (敵単体へのダメージ)
        #-----------------------------
        
        if self.role == HERO and target and not isinstance(target[0], Explorer):
            other = target[0] 
            damage = self.power * 1.5 + random.randint(-3, 3)
            other.hp -= damage
            
            if other.hp <= 0:
                other.hp = 0
                results.append(f"{other.name}に{damage}ダメージ！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!")
            else:
                results.append(f"{other.name}に{damage}ダメージ！{other.name}の体力は{other.hp}になった。")
            
            return f"{self.name}の「閃空斬り」発動！\n" + "".join(results)
        
        #--------------------------
        #ヒーラーの必殺技: 女神の加護(味方の回復)
        #--------------------------
        
        elif self.role == HEALER and target:
            for other in target:
                # 生きている味方かつExplorerのみを回復対象とする(死んだやつを対象にする変更も考慮)
                if other.is_alive() and isinstance(other, Explorer):
                    heal_amount = 25 + random.randint(0, 10)
                    other.hp = min(other.max_hp, other.hp + heal_amount)
                    results.append(f"{other.name}を{heal_amount}回復！HPは{other.hp}になった。")
            
            return f"{self.name}の「女神の加護」発動！\n" + "\n".join(results)

        #------------------------------
        # 魔法使いの必殺技: マジックゲイン (味方全体MP回復)
        #------------------------------
        
        elif self.role == MAGICIAN and target:
            mp_recover = 3
            for other in target:
                # 生きている味方かつExplorerのみを回復対象とする
                if other.is_alive() and isinstance(other, Explorer):
                    other.mp += mp_recover
                    results.append(f"{other.name}のMPを{mp_recover}回復！MPは{other.mp}になった。")
            
            return f"{self.name}の「マジックゲイン」発動!\n" + "\n".join(results)
        
        # ターゲット選択が不適切だった場合など
        return f"{self.name}: 必殺技のターゲットが不適切です。"

    
    def leader_skill(self, party_members): 
        # 自身がリーダーとして設定されていない場合は発動しない
        if not self.leader:
            return f"{self.name}はリーダーではないため、スキルは発動しません。\n"
        
        results = []

        # 勇者: 攻撃力アップ
        if self.role == HERO:
            for member in party_members:
                # powerを1.2倍し、整数に切り上げ
                new_power = math.ceil(member.power * 1.2)
                results.append(f"└ {member.name}の攻撃力が {member.power} から {new_power} にアップ!")
                member.power = new_power
                
        # ヒーラー: 最大HPアップ (現在のHPも合わせて増加)
        elif self.role == HEALER:
            for member in party_members:
                # max_hpを1.2倍し、整数に切り上げ
                hp_increase = math.ceil(member.max_hp * 0.2)
                member.max_hp += hp_increase
                member.hp += hp_increase # 現在のHPも最大値と同じだけ増加させる
                results.append(f"{member.name}の最大HPが {member.max_hp - hp_increase} から {member.max_hp} にアップ!")
                
        # 魔法使い: MPアップ (最大MP属性がない場合はmpを直接増加)
        elif self.role == MAGICIAN:
            # Characterクラスにmax_mpがないと仮定し、mpを直接増加させるが、本来はmax_mpが必要
            for member in party_members:
                # mpを1.2倍し、整数に切り上げ
                mp_increase = math.ceil(member.max_mp * 0.2)
                member.max_mp += mp_increase
                member.mp += mp_increase
                results.append(f"{member.name}の最大MPが {member.max_mp - mp_increase} から {member.max_mp} にアップ!")
        
        return f"{self.name}のリーダースキル発動！\n" + "\n".join(results)
        
        

    def reset(self):
        """初期状態に戻す"""
        self.__dict__ = copy.deepcopy(self._initial_state)

In [41]:
#モンスターのクラス定義
import copy
import random 

DRAGON = 'dragon'
HARPY = 'harpy'
WITCH = 'witch'

class Monster(Character):
    def __init__(self, name, hp, power, mp, breed):
        super().__init__(name, hp, power, mp)
        self.breed = breed 
        self._initial_state = copy.deepcopy(self.__dict__)

    # Characterクラスのspecial_attackをオーバーライド
    def special_attack(self, target):
        results = []
        
        # ターゲットが単体の場合も考慮し、リストに変換
        if not isinstance(target, list):
            target = [target]

        #-----------------------------------
        # ドラゴンの必殺技 (敵全体攻撃)
        #-----------------------------------
        if self.breed == DRAGON:
            if self.mp < 3:
                return f"{self.name}: MPが足りない！ (現在のMP: {self.mp})"
            self.mp -= 3
            
            damage_base = self.power * 2
            
            # ターゲットリスト全体をループ
            for other in target:
                damage = damage_base + random.randint(-5, 5)
                other.hp -= damage
                
                if other.hp <= 0:
                    other.hp = 0
                    results.append(f"{other.name}に{damage}ダメージ！{other.name}は{self.name}に倒された!")
                else:
                    results.append(f"{other.name}に{damage}ダメージ！{other.name}の体力は{other.hp}になった。")
                    
            return f"{self.name}のファイヤーブレス発動!\n" + "\n".join(results)
            
        #---------------------------------
        # ハーピーの必殺技: 奇跡の歌声 (maxHPの50%自己回復+相手の攻撃力を10パーセント下げる)
        #----------------------------------
        elif self.breed == HARPY:
            if self.mp < 3:
                return f"{self.name}: MPが足りない！ (現在のMP: {self.mp})"
            self.mp -= 3
            heal_amount = self.max_hp*0.5 + random.randint(-5, 10)
            self.hp = min(self.max_hp, self.hp + heal_amount)
            
            # ターゲットリスト全体をループ
            for other in target:
                debuf = (10 + random.randint(-2, 5)) / 100
                debuf_amount = math.ceil(other.power * debuf)
                other.power -= debuf_amount
                if other.power <= 0:
                    other.power = 0
                    results.append(f"{other.name}の攻撃力が{debuf_amount}%減って{other.power}に下がった！")
                else:
                    results.append(f"{other.name}の攻撃力が{debuf_amount}%減って{other.power}に下がった！")
            return f"{self.name}の「奇跡の歌声」発動！\n{self.name}は体力を{heal_amount}回復！体力は{self.hp}になった。\n" + "\n".join(results)

        #---------------------------------------------
        # ウィッチの必殺技:マジカルドレイン (敵全体MPドレイン)
        #---------------------------------------------
        elif self.breed == WITCH:
            if self.mp < 3:
                return f"{self.name}: MPが足りない！ (現在のMP: {self.mp})"
            self.mp -= 3
            
            drained_mp_total = 0
            
            # ターゲットリスト全体をループ
            for other in target:
                mp_drain = 2 + random.randint(-1, 3)
                drained_mp_total += mp_drain
                other.mp = max(0, other.mp - mp_drain)
                results.append(f"{other.name}のMPを{mp_drain}奪った！MPは{other.mp}になった。")
                
            self.power += drained_mp_total # 奪ったMPの総量を自身の攻撃力に追加
            
            return f"{self.name}のドレイン発動！\n" + "\n".join(results) + f"\n{self.name}の攻撃力が{drained_mp_total}上がった！"
        
        # どのbreedにも一致しなかった場合の処理
        return self.attack(target) # 定義外の種族は通常攻撃にフォールバック

        #特殊スキル
        #後々実装

In [42]:
#バトルのマネジメント
import random
import sys

# 定数はCharacter/Explorer/Monsterクラスで定義済みのものを使用
HERO = 'hero'
HEALER = 'healer'
MAGICIAN = 'magician'
DRAGON = 'dragon'
HARPY = 'harpy'
WITCH = 'witch'


class BattleManager:
    def __init__(self, party, enemy_team):
        self.party = party
        self.enemy_team = enemy_team
        self.turn = 1
        self.is_game_over = False
        
        # 戦闘開始時にリーダースキルを発動
        self._activate_leader_skill()

    #-------------パーティ内でリーダーが設定されていればスキルを発動する-------
    def _activate_leader_skill(self):
        leader_char = next((p for p in self.party if p.leader), None)

        if leader_char:
            #Explorerのleader_skillメソッドにパーティメンバー全体を渡す
            print("\n" + "="*50)
            print(leader_char.leader_skill(self.party)) 
            print("="*50)
        else:
            print("\nリーダーは設定されていません。リーダースキルは発動しません。")

    #-----------------全員のステータスを表示-------------------------------
    def display_status(self):
        print("\n" + "="*50)
        print(f"--- ターン: {self.turn} ---")
        
        # パーティーステータス
        print("\n[勇者パーティ]")
        for i, member in enumerate(self.party):
            leader_tag = " (リーダー)" if member.leader else ""
            print(f" {i+1}. {member.name} ({member.role}{leader_tag}): HP {member.hp}/{member.max_hp}, MP {member.mp}/{member.max_mp}, 攻撃力 {member.power}")
            
        
        # モンスター隊ステータス (1体のみ)
        print("\n[モンスター]")
        for i, monster in enumerate(self.enemy_team):
            print(f" {i+1}. {monster.name} ({monster.breed}): HP {monster.hp}/{monster.max_hp}, MP {monster.mp}/{monster.max_mp}, 攻撃力 {monster.power}")
        print("="*50 + "\n")

    #-------------------ユーザーにターゲットを単体選択させるヘルパー関数-------------------
    def _select_target(self, targets, action_type="攻撃"):
        while True:
            try:
                print(f"{action_type}ターゲットを選択してください:")
                
                # 生きているターゲットのみ表示
                alive_targets = [t for t in targets if t.is_alive()]
                if not alive_targets:
                    print("ターゲットがいません！")
                    return None

                for i, target in enumerate(alive_targets):
                    print(f" {i+1}. {target.name} (HP: {target.hp})")
                
                target_index = int(input("番号を入力 (1/...): ")) - 1
                
                if 0 <= target_index < len(alive_targets):
                    # 選択された単体ターゲットを返す
                    return alive_targets[target_index]
                else:
                    print("無効な番号です。再入力してください。")
            except ValueError:
                print("数字を入力してください。")

    #---------------プレイヤーが操作するターン-------------------------------
    def player_turn(self):
        print("--- プレイヤーのターン ---")
        
        for player in self.party:
            if not player.is_alive():
                continue

            print(f"\n[{player.name} の行動]")
            
            # 生きている敵と味方のリストを作成
            alive_enemies = [m for m in self.enemy_team if m.is_alive()]
            alive_allies = [p for p in self.party if p.is_alive()]
            
            if not alive_enemies:
                return # 敵がいないためターン終了

            while True:
                action = input(f"行動を選択 (a:通常攻撃 / s:必殺技): ").lower()
                
                # --- 通常攻撃 (全体攻撃) ---
                if action == 'a':
                    # Character.attackはリストを受け入れるため、生きている敵全体を渡す
                    target_list = alive_enemies
                    print(player.attack(target_list))
                    break
                
                # --- 必殺技 ---
                elif action == 's':
                    if player.mp < 3:
                        print(f"{player.name}のMPが足りません")
                        continue
                        
                    if player.role == HERO:
                        #勇者の場合は単体ターゲットを選択させる
                        target = self._select_target(alive_enemies, "勇者の必殺技")
                        if not target: continue
                        target_list = [target] # リスト化してspecial_attackに渡す
                        
                    elif player.role == HEALER:
                        # ヒーラー/魔法使いは味方全体をターゲット
                        target_list = alive_allies

                    elif player.role == MAGICIAN:
                        #魔法使いの対象は自分以外全員
                        target_list = [m for m in alive_allies if m.role != MAGICIAN]
                        
                    else:
                        print("その役割には必殺技がありません。")
                        continue
                    
                    print(player.special_attack(target_list))
                    break
                
                else:
                    print("無効な入力です。'a' または 's' を入力してください。")
            
            if self.check_game_status():
                return 

    #--------------------------モンスターが行動するターン (簡単なAI)------------------------------
    def enemy_turn(self):
        print("\n--- モンスターのターン ---")
        
        alive_players = [p for p in self.party if p.is_alive()]
        if not alive_players:
            return

        # モンスターは1体のみ
        monster = self.enemy_team[0] 
        if not monster.is_alive():
            return
        
        # ターゲット決定
        monster_breed = monster.breed
        
        # モンスターのAI: 必殺技 or 通常攻撃
        # AIロジック: 40%の確率、またはMPが3以上ありHPが半分以下の場合は必殺技
        use_special_attack = (random.random() < 0.4) or \
                             (monster.mp >= 3 and monster.hp < monster.max_hp * 0.5)

        if use_special_attack:
            # 必殺技を使用する場合のターゲット設定
            if monster_breed == DRAGON or monster_breed == WITCH:
                # ドラゴン(全体攻撃)、ウィッチ(全体ドレイン)はパーティ全体をターゲット
                target_list = alive_players 
            elif monster_breed == HARPY:
                target_list = alive_players
            else:
                 # 未定義の場合に備えて
                target_list = alive_players

            print(monster.special_attack(target_list))

        else:
            # 通常攻撃の場合、Character.attackはリストを受け入れるため、パーティ全体を渡す
            target_list = alive_players
            print(monster.attack(target_list))

        if self.check_game_status():
            return
            
    #---------------------勝敗のチェック---------------------------------
    def check_game_status(self):
        alive_party = [p for p in self.party if p.is_alive()]
        alive_enemies = [m for m in self.enemy_team if m.is_alive()]
        
        if not alive_party:
            print("\n!!! 勇者パーティは全滅した。ゲームオーバー !!!")
            self.is_game_over = True
            return True
        
        if not alive_enemies:
            print("\n!!! モンスターをすべて倒した！勝利 !!!")
            self.is_game_over = True
            return True
        
        # 生きているメンバーを更新
        self.party = alive_party
        self.enemy_team = alive_enemies
        return False


    #------------------------戦闘の開始と実行--------------------------------------------
    def start_battle(self):
        enemy_name = self.enemy_team[0].name if self.enemy_team else "敵"
        print(f"{enemy_name}が立ちはだかる！バトル開始！")
        
        while not self.is_game_over:
            self.display_status()
            
            # プレイヤーターン
            self.player_turn()
            if self.is_game_over: break
            
            # モンスターターン
            self.enemy_turn()
            if self.is_game_over: break
            
            self.turn += 1
        
        print("\n--- 戦闘終了 ---")

In [40]:
# --- メイン実行部分 ---
if __name__ == "__main__":

    print("各キャラクターの情報は下を確認！")
    
    #----------既存のパーティメンバー情報----------
    initial_party_info = [
        ("アーサー", 100, 15, 5, HERO, 1),
        ("リア", 80, 10, 8, HEALER, 1),
        ("マーリン", 70, 17, 10, MAGICIAN, 1),
    ]

    party_members = []
    
    #------------ユーザーによるリーダー設定-------------
    print("\n--- リーダー設定  ---")
    
    #選択肢の表示
    for i, (name, hp, power, mp, role, level) in enumerate(initial_party_info):
        print(f" {i+1}. {name} ({role})")

    while True:
        try:
            choice = input("リーダーにしたいメンバーの番号を入力してください (1/2/3): ")
            leader_index = int(choice) - 1

            if 0 <= leader_index < len(initial_party_info):
                # 選択されたメンバーをリーダーに設定し、インスタンスを生成
                for i, info in enumerate(initial_party_info):
                    name, hp, power, mp, role, level = info
                    is_leader = (i == leader_index)
                    
                    # Explorerインスタンスを生成
                    member = Explorer(name, hp, power, mp, role, level, leader=is_leader)
                    party_members.append(member)
                
                print(f"{party_members[leader_index].name} をリーダーに設定しました。")
                break
            else:
                print("無効な番号です。1から3の番号を入力してください。")
        except ValueError:
            print("数字を入力してください。")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            sys.exit(1)
    
    #----モンスター隊の生成(ランダムに1体を選択)--------------
    MONSTER_TEMPLATES = {
        DRAGON: ("フレイムドラゴン", 300, 22, 6), 
        HARPY: ("セイレーン", 260, 18, 9),   
        WITCH: ("リリス", 280, 20, 12),   
    }

    random_breed = random.choice([DRAGON, HARPY, WITCH])
    name, hp, power, mp = MONSTER_TEMPLATES[random_breed]
    
    single_monster = Monster(name, hp, power, mp, random_breed)
    enemy_squad = [single_monster]

    #----------バトル開始------------------------------
    try:
        game = BattleManager(party_members, enemy_squad)
        game.start_battle()
    except EOFError:
        print("\nゲームを終了します。")
    except Exception as e:
        print(f"\n予期せぬエラーが発生しました: {e}")
        sys.exit(1)

各キャラクターの情報は下を確認！

--- リーダー設定  ---
 1. アーサー (hero)
 2. リア (healer)
 3. マーリン (magician)


リーダーにしたいメンバーの番号を入力してください (1/2/3):  1


アーサー をリーダーに設定しました。

アーサーのリーダースキル発動！
└ アーサーの攻撃力が 15 から 18 にアップ!
└ リアの攻撃力が 10 から 12 にアップ!
└ マーリンの攻撃力が 17 から 21 にアップ!
セイレーンが立ちはだかる！バトル開始！

--- ターン: 1 ---

[勇者パーティ]
 1. アーサー (hero (リーダー)): HP 100/100, MP 5/5, 攻撃力 18
 2. リア (healer): HP 80/80, MP 8/8, 攻撃力 12
 3. マーリン (magician): HP 70/70, MP 10/10, 攻撃力 21

[モンスター]
 1. セイレーン (harpy): HP 260/260, MP 9/9, 攻撃力 18

--- プレイヤーのターン ---

[アーサー の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  s


勇者の必殺技ターゲットを選択してください:
 1. セイレーン (HP: 260)


...

マーリンがセイレーンを攻撃！セイレーンのHPは16になった。


--- モンスターのターン ---
セイレーンがアーサーを攻撃！アーサーのHPは44になった。
セイレーンがリアを攻撃！リアのHPは26になった。
セイレーンがマーリンを攻撃！マーリンのHPは19になった。


--- ターン: 15 ---

[勇者パーティ]
 1. アーサー (hero (リーダー)): HP 44/100, MP 2/5, 攻撃力 14
 2. リア (healer): HP 26/80, MP 2/8, 攻撃力 9
 3. マーリン (magician): HP 19/70, MP 1/10, 攻撃力 17

[モンスター]
 1. セイレーン (harpy): HP 16/260, MP 0/9, 攻撃力 18

--- プレイヤーのターン ---

[アーサー の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  a


アーサーがセイレーンを攻撃！セイレーンのHPは4になった。


[リア の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  a


リアがセイレーンを攻撃！セイレーンのHPは0になった。
セイレーンはリアに倒された!


!!! モンスターをすべて倒した！勝利 !!!

--- 戦闘終了 ---


---

#### ・パーティーキャラクター情報

```
・アーサー(勇者): 必殺技「閃空斬り」・・・自身の攻撃力の2倍のダメージを敵一体に与える。(消費MP: 3)

・リア(ヒーラー): 必殺技「女神の加護」・・・味方全員のHPを40回復。(消費MP: 3)

・マーリン(魔法使い): 必殺技「マジックゲイン」・・・自分以外の全員のMPを3回復。(消費MP: 3)
```

---

#### ・モンスター情報

```
・フレイムドラゴン: 必殺技「ファイヤーブレス」: 敵全員に自身の攻撃力の2.7倍のダメージを与える。(消費MP: 3)
  
・セイレーン: 必殺技:「奇跡の歌声」: 自身の最大HPの半分回復し、相手全員の攻撃値を３パーセント下げる (消費MP: 3)

・ リリス: 必殺技「マジカルドレイン」: 相手全員のMPを2ずつ吸収する(消費MP 3)
```

---

[補足]
攻撃には乱数が存在し、数値は約1~5パーセント程度上下します。